In [ ]:
import _pickle as cPickle
from numpy.random import seed
seed(2023)
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib widget

from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import find_peaks, savgol_filter

import mrmr
from mrmr import mrmr_classif

We import the dataset used for this test: **RadioML 2016.10A**.
<br>
This dataset can be found in https://www.deepsig.ai/datasets/

In [ ]:
Xd = pd.read_pickle("/home/ymondino/Documents/repos/machinelearning/RML2016/RML2016.10a_dict.pkl")

The variables **snrs** and **mods** contain all the SNR values and modulation types that exist in the dataset.

In [ ]:
snrs, mods = map(lambda j: sorted(set(map(lambda x: x[j], Xd.keys()))), [1,0])


We do not make use of all SNR values and modulation types, so **usedSnr** and **usedMods** contain the used ones for training, testing and validating the model.

In [ ]:
usedMods = ['BPSK','QAM16','QPSK','8PSK','GFSK','PAM4','CPFSK']
usedSnr = [-10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

The variable **X** contains the signals that will be used for training, testing and validating the model. While **lbl** contains the information about the modulation type and SNR value of each of these signals.

In [ ]:
X = []  
lbl = []

for mod in usedMods:
    for snr in usedSnr:
        X.append(Xd[(mod,snr)])  
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
            
X = np.vstack(X)
X.shape

**X_mods** contains only the information about modulation type of each signal example.

In [ ]:
X_mods = np.array(list(map(lambda x: lbl[x][0],range(0,len(lbl)))))
len(X_mods)

Each combination of SNR value and modulation type has 1000 signal examples, we choose 600 of these 1000 signals to be used for testing the model.

In [ ]:
#n_examples = 1000

#n_test = 600       

#test_idx = np.random.choice(list(set(range(0, n_examples))), size = n_test, replace = False)

#test_idx[0:5]

**dataset** contains the identification of the 400 signal examples, of each combination of SNR value and modulation type, used for training and validating the model.

In [ ]:
#dataset = list(set(range(0, n_examples)) - set(test_idx))
#len(dataset)

**calcAllFeatures** calculates 32 different features that have been used in previous classification algorithms.
<br><br>
PARAMETERS:
    <br>- **X**: Signal examples.
    <br>- **idx**: Indexes of the signal examples to consider from all the ones contained in X.
    <br>- **X_mods**: Modulation types of the signal examples in X.
    <br>- **lbl**: Information of modulation type and SNR value of the signal examples in X.
    <br>- **usedSnr**: List of SNR values present in X.
<br><br>
RETURNS:
    <br>- 32 arrays (one for each feature) with the calculated feature values for each of the signal examples.

In [ ]:
def calcAllFeatures (X, idx, mods, X_mods, lbl, usedSnr):
      
    n_train = len(idx)
    n_snr = len(usedSnr)

    sigma_ap_values = np.zeros((len(mods) * n_snr, n_train))
    sigma_a_values = np.zeros((len(mods) * n_snr, n_train))
    sigma_aa_values = np.zeros((len(mods) * n_snr, n_train))
    sigma_dp_values = np.zeros((len(mods) * n_snr, n_train))
    sigma_af_values = np.zeros((len(mods) * n_snr, n_train))
    sigma_deltap_values = np.zeros((len(mods) * n_snr, n_train))
    sigma_v_values = np.zeros((len(mods) * n_snr, n_train))
    
    C20_values = np.zeros((len(mods) * n_snr, n_train))
    C21_values = np.zeros((len(mods) * n_snr, n_train))
    
    C40_values = np.zeros((len(mods) * n_snr, n_train)) 
    C41_values = np.zeros((len(mods) * n_snr, n_train)) 
    C42_values = np.zeros((len(mods) * n_snr, n_train))
    
    C60_values = np.zeros((len(mods) * n_snr, n_train))
    C61_values = np.zeros((len(mods) * n_snr, n_train))
    C62_values = np.zeros((len(mods) * n_snr, n_train))
    C63_values = np.zeros((len(mods) * n_snr, n_train))
    
    C80_values = np.zeros((len(mods) * n_snr, n_train))
    C81_values = np.zeros((len(mods) * n_snr, n_train))
    C84_values = np.zeros((len(mods) * n_snr, n_train))
    
    v20_values = np.zeros((len(mods) * n_snr, n_train)) 
    v30_values = np.zeros((len(mods) * n_snr, n_train)) 
    
    pa_values = np.zeros((len(mods) * n_snr, n_train))
    kurtosis_values = np.zeros((len(mods) * n_snr, n_train))
    kurtosis_f_values = np.zeros((len(mods) * n_snr, n_train))
    beta_values = np.zeros((len(mods) * n_snr, n_train)) 
    skewness_values = np.zeros((len(mods) * n_snr, n_train)) 
    skewness_f_values = np.zeros((len(mods) * n_snr, n_train)) 
    
    mean_amp_values = np.zeros((len(mods) * n_snr, n_train))
    std_amp_values = np.zeros((len(mods) * n_snr, n_train))
    mean_temp_values = np.zeros((len(mods) * n_snr, n_train))
    std_temp_values = np.zeros((len(mods) * n_snr, n_train))
    
    gamma_max_values = np.zeros((len(mods) * n_snr, n_train))
    gamma_2_values = np.zeros((len(mods) * n_snr, n_train))
    
    rms_ratio_values = np.zeros((len(mods) * n_snr, n_train))
    avg_ratio_values = np.zeros((len(mods) * n_snr, n_train))
    
    spectral_var_values = np.zeros((len(mods) * n_snr, n_train))
    deviation_values = np.zeros((len(mods) * n_snr, n_train))
   
    for i,mod in enumerate (mods): 
        idx_sameMod = np.where(np.array(X_mods) == mod)[0]
        X_sameMod = X[idx_sameMod]  
        
        for j,snr in enumerate (usedSnr):  
            
            snr_sameMod = np.array(list(map(lambda x: lbl[x][1], idx_sameMod)))
            idx_sameMod_snr = np.where(np.array(snr_sameMod == snr))[0]
            X_sameMod_snr = X_sameMod[idx_sameMod_snr]   
            
            X_idx = X_sameMod_snr[idx]

            for ex in range(0, X_idx.shape[0]):
                
                X_0 = X_idx[ex][0]
                X_1 = X_idx[ex][1]
                
                signal = X_0 + X_1 * 1j
                
                # The next normalization is the one implemented on the public dataset.
                # We implement it in the same way to normalize the cases when the signals are interfered
                energy = np.sum(np.abs(signal))
                signal = signal / np.sqrt(energy)
                
                ampli = abs(signal) 
                
                X_0 = np.real(signal)
                X_1 = np.imag(signal)
                
                ma = ampli.mean()
                
                acn = ampli / ma - 1
                
                #---------------------
                
                angle = np.angle(signal)
                
                sigma_dp_values[j + i*n_snr][ex] = angle.std()
                
                #---------------------
                
                angle = abs(angle)
                
                sigma_ap_values[j + i*n_snr][ex] = angle.std()
                
                #---------------------
                
                acn2 = abs(acn)
                
                sigma_aa_values[j + i*n_snr][ex] = acn2.std()
                
                #---------------------
                
                frec = 1 / (2 * np.pi) * 1/(1 + (X_1 / X_0)**2)
                
                mf = frec.mean()
                
                frec = frec / mf - 1
                
                frec = abs(frec)
                
                sigma_af_values[j + i*n_snr][ex] = frec.std()
                
                #---------------------
            
                
                sigma_a_values[j + i*n_snr][ex] = acn.std()
                
                #---------------------
                
                frec = 1 / (2 * np.pi) * 1 / (1 + (X_1 / X_0)**2)
                
                sigma_deltap_values[j + i*n_snr][ex] = frec.std()
                
                #---------------------
                
                a = np.sqrt(ampli / ampli.var()) - 1
                
                
                sigma_v_values[j + i*n_snr][ex] = a.std()
                
                #---------------------
                
                C20_values[j + i*n_snr][ex] = abs((signal**2).mean())
                
                #---------------------
                
                C21_values[j + i*n_snr][ex] = (ampli**2).mean()
                
                #---------------------
                
                M20 = (signal**2).mean()
                
                M21 = (signal * np.conj(signal)).mean()
                
                M22 = (np.conj(signal)**2).mean()
                
                M40 = (signal**4).mean()
                
                M41 = (signal**3 * np.conj(signal)).mean()
                
                M42 = (signal**2 * np.conj(signal)**2).mean()
                
                M43 = (signal * np.conj(signal)**3).mean()
                
                M44 = (np.conj(signal)**4).mean()
                
                M62 = (signal**4 * np.conj(signal)**2).mean()
                
                M63 = (signal**3 * np.conj(signal)**3).mean()
                
                M60 = (signal**6).mean()

                M80 = (signal**8).mean()
                
                M84 = (signal**4 * np.conj(signal)**4).mean()
                
                M64 = (signal**2 * np.conj(signal)**4).mean()

                #---------------------
                
                C40_value = M40 - 3 * M20**2
                
                C40_values[j + i*n_snr][ex] = abs(C40_value)**(2 / 4)
                
                #---------------------
                
                C41_value = M41 - 3 * M20 * M21
                
                C41_values[j + i*n_snr][ex] = abs(C41_value)**(2 / 4)
                
                #---------------------
                
                C42_value = M42 -  abs(M20)**2 - 2 * M21**2
                
                C42_values[j + i*n_snr][ex] = abs(C42_value)**(2 / 4) 
                
                #---------------------
                
                C60_value = M60 - 15 * M20 * M40 + 3 * M20**3
                
                C60_values[j + i*n_snr][ex] = abs(C60_value)**(2 / 6)
                
                #---------------------
                
                M61 = (signal**5 * np.conj(signal)).mean()
                
                C61_value = M61 - 5 * M21 * M40 - 10 * M20 * M41 + 30 * M20**2 * M21
                
                C61_values[j + i*n_snr][ex] = abs(C61_value)**(2 / 6)
                
                #---------------------
                
                C62_value = M62 - 6 * M20 * M42 - 8 * M21 * M41 - M22 * M40 \
                            + 6 * M20**2 * M22 + 24 * M21**2 * M20
                
                C62_values[j + i*n_snr][ex] = abs(C62_value)**(2 / 6)
                
                #---------------------
                
                C63_value = M63 - 9 * M42 * M21 + 12 * M21**3 - 3 * M20 * M43\
                            - 3 * M22 * M41 + 18 * M20 * M21 * M22 
                
                C63_values[j + i*n_snr][ex] = abs(C63_value)**(2 / 6)
                
                #---------------------
                
                C80_value = M80 - 35 * M40**2 - 28 * M60 * M20 \
                            + 420 * M20**2 * M40 - 630 * M20**4
                
                C80_values[j + i*n_snr][ex] = abs(C80_value)**(2 / 8)
                
                #---------------------
                
                M81 = (signal**7 * np.conj(signal)).mean()
                
                C81_value = M81 - 21 * M20 * M61 - 7 * M21 * M60 - 35 * M40 * M41 \
                            + 210 * M20**2 * M41 + 210 * M20 * M21 * M40 \
                            - 630 * M20**3 * M21

                C81_values[j+i*n_snr][ex] = abs(C81_value)**(2/8)
                
                #---------------------
                
                C84_value = M84 - 6 * M20 * M64 - 16 * M21 * M63 \
                            - 6 * M22 * M62 - M40 * M44 - 16 * M41 * M43 \
                            - 18 * M42**2 + 6 * M20**2 * M44 + 96 * M20 * M21 * M43 \
                            + 72 * M20 * M22 * M42 + 144 * M21**2 * M42 \
                            + 96 * M21 * M22 * M41 + 6 * M22*2 * M40 - 54 * M20**2 * M22**2 \
                            - 432 * M20 * M21**2 * M22 - 144 * M21**4

                C84_values[j + i*n_snr][ex] = abs(C84_value)**(2 / 8)
                
                #---------------------
                
                v20_values[j + i*n_snr][ex] = np.real(M42 / M21**2)
                
                #---------------------
                
                v30_values[j + i*n_snr][ex] = abs(M63 / M21**3)
                
                #---------------------
                
                max_a = max(abs(ampli))
                
                ma = ampli.mean()
                
                avg_ratio_values[j + i*n_snr][ex] = max_a / ma
                
                #---------------------
                
                rms = (ampli**2).mean()
                
                rms_ratio_values[j + i*n_snr][ex] = max_a**2 / rms
                
                #--------------------
                
                acn = ampli / ma - 1
                
                kurtosis_values[j + i*n_snr][ex] = (acn**4).mean() /  (acn**2).mean()

                #-------------------
                
                frec = 1 / (2 * np.pi) * 1 / (1 + (X_1 / X_0)**2)
                
                mf = frec.mean()
                sf = frec.std()
                
                frec = frec / mf - 1
                
                kurtosis_f_values[j + i*n_snr][ex] = (frec**4).mean() / (frec**2).mean()
                
                #-------------------
                
                ai_2 = X_0**2
                aq_2 = X_1**2
                
                beta_values[j + i*n_snr][ex] = sum(aq_2) / sum(ai_2)
                
                #-------------------
                
                num = ((ampli - ma)**3).mean()
                
                den = ((ampli - ma)**2).mean()
                den = den**(3 / 2)
                
                skewness_values[j + i*n_snr][ex] = abs(num / den)
                
                #-------------------
                
                R = fft(signal)
                R = fftshift(R)
                
                mu = R.mean()
                
                sigma = ((R - mu) * np.conj(R - mu)).mean()
                
                num = ((R - mu)**3).mean()
                
                den = sigma**(3 / 2)
                
                skewness_f_values[j + i*n_snr][ex] = abs(num / den)
                
                #--------------------
                
                N = len(ampli)
                
                dft = fft(acn)
                
                gamma_max_values[j + i*n_snr][ex] = np.max(np.abs(dft)**2) / N
            
                #--------------------
                
                dft = fft(ampli**2)
                 
                N = len(ampli)
                
                gamma_2_values[j + i*n_snr][ex] = np.max(np.abs(dft)**2) / N
                
                #--------------------

                dft = np.abs(np.fft.fftshift(np.fft.fft(signal,n = 64)))

                n = len(dft)   
                dx = 1 / 1e6            
                frec_s = np.fft.fftshift(np.fft.fftfreq(n, dx))

                F = []
                for fi in range(len(frec_s) - 1):
                       F.append((dft[fi + 1] - dft[fi]) / (abs(frec_s[fi + 1] - frec_s[fi])))

                spectral_var_values[j + i*n_snr][ex] = np.var(F)
            
                #---------------------
            
                deviation_values[j + i*n_snr][ex] = np.var(abs(signal) / (abs(signal)).mean() - 1)
               
                #--------------------
                
                X0_f = savgol_filter(X_0, 7, 2)
                X1_f = savgol_filter(X_1, 7, 2)

                ampli = np.sqrt(np.square(X0_f) + np.square(X1_f)) 
                
                peaks, _ = find_peaks(ampli, distance = 2) 
                negPeaks,_ = find_peaks(-ampli, distance = 2)
    
                allPeaks = np.concatenate((peaks, negPeaks,[0]))
                allPeaks = np.sort(allPeaks)

                ampSeg = []
                tempSeg = []

                for indx in range(0, len(allPeaks) - 1):
                    ampSeg.append(abs(ampli[allPeaks[indx + 1]] - ampli[allPeaks[indx]]))
                    tempSeg.append(allPeaks[indx + 1] - allPeaks[indx])
                    
                ampSeg = np.array(ampSeg)
     
                mean_amp_values[j + i*n_snr][ex] = ampSeg.mean()
        
                tempSeg = np.array(tempSeg)
     
                mean_temp_values[j + i*n_snr][ex] = tempSeg.mean()
        
                #-------------------
            
                ampSeg = ampSeg - ampSeg.mean()
     
                std_amp_values[j + i*n_snr][ex] = sum(abs(ampSeg)) / len(ampSeg)
        
                
                tempSeg = tempSeg - tempSeg.mean()
     
                std_temp_values[j + i*n_snr][ex] = sum(abs(tempSeg)) / len(tempSeg)
        
                
                
    return sigma_a_values, sigma_ap_values, sigma_aa_values, sigma_dp_values, \
        sigma_af_values, sigma_deltap_values,  sigma_v_values, \
        C20_values, C21_values, C40_values, C41_values, C42_values, \
        C60_values, C61_values, C62_values, C63_values, \
        C80_values, C81_values, C84_values, v20_values, v30_values, \
        kurtosis_values, kurtosis_f_values, beta_values, skewness_values, \
        skewness_f_values, mean_amp_values, std_amp_values, mean_temp_values, \
        std_temp_values, gamma_max_values, gamma_2_values, rms_ratio_values, \
        avg_ratio_values, spectral_var_values, deviation_values



**generateAllDataArray** generates the input data array of the neural network.
<br><br>
PARAMETERS:
    <br>- **X**: Signal examples.
    <br>- **idx**: Indexes of the signal examples to consider from all the ones contained in X.
    <br>- **X_mods**: Modulation types of the signal examples in X.
    <br>- **lbl**: Information of modulation type and SNR value of the signal examples in X.
    <br>- **usedSnr**: List of SNR values present in X.
<br><br>
RETURNS:
    <br>- One matrix that contains all the input features of the neural network.

In [ ]:
def generateAllDataArray(X, idx, mods, X_mods, lbl, usedSnr):
  
    sigma_a_values,sigma_ap_values, sigma_aa_values, \
    sigma_dp_values, sigma_af_values, sigma_deltap_values,  \
    sigma_v_values, C20_values, C21_values, C40_values, \
    C41_values, C42_values, C60_values, C61_values, \
    C62_values, C63_values, C80_values, C81_values, \
    C84_values, v20_values, v30_values, kurtosis_values, \
    kurtosis_f_values, beta_values, skewness_values, \
    skewness_f_values, mean_amp_values, std_amp_values, \
    mean_temp_values, std_temp_values, gamma_max_values, \
    gamma_2_values, rms_ratio_values, avg_ratio_values, \
    spectral_var_values, deviation_values  = calcAllFeatures (X, idx, mods, X_mods, lbl, usedSnr)    
  
        
    X_all = np.hstack(( \
                        sigma_a_values[0].reshape(-1, 1), \
                        sigma_ap_values[0].reshape(-1, 1), \
                        sigma_aa_values[0].reshape(-1, 1), \
                        sigma_dp_values[0].reshape(-1, 1), \
                        sigma_af_values[0].reshape(-1, 1), \
                        sigma_deltap_values[0].reshape(-1, 1), \
                        sigma_v_values[0].reshape(-1, 1), \
                        C20_values[0].reshape(-1, 1), \
                        C21_values[0].reshape(-1, 1), \
                        C40_values[0].reshape(-1, 1), \
                        C41_values[0].reshape(-1, 1), \
                        C42_values[0].reshape(-1, 1), \
                        C60_values[0].reshape(-1, 1), \
                        C61_values[0].reshape(-1, 1), \
                        C62_values[0].reshape(-1, 1), \
                        C63_values[0].reshape(-1, 1), \
                        C80_values[0].reshape(-1, 1), \
                        C81_values[0].reshape(-1, 1), \
                        v20_values[0].reshape(-1, 1), \
                        v30_values[0].reshape(-1, 1), \
                        kurtosis_values[0].reshape(-1, 1), \
                        kurtosis_f_values[0].reshape(-1, 1), \
                        beta_values[0].reshape(-1, 1), \
                        skewness_values[0].reshape(-1, 1), \
                        skewness_f_values[0].reshape(-1, 1), \
                        mean_amp_values[0].reshape(-1, 1), \
                        std_amp_values[0].reshape(-1, 1), \
                        mean_temp_values[0].reshape(-1, 1), \
                        std_temp_values[0].reshape(-1, 1), \
                        gamma_max_values[0].reshape(-1, 1), \
                        gamma_2_values[0].reshape(-1, 1), \
                        rms_ratio_values[0].reshape(-1, 1), \
                        avg_ratio_values[0].reshape(-1, 1) ,\
                        spectral_var_values[0].reshape(-1, 1) ,\
                        deviation_values[0].reshape(-1, 1) \
                      ))


    for p in range(1,len(usedSnr)*len(mods)):
        X_data = np.hstack((\
                            sigma_a_values[p].reshape(-1, 1), \
                            sigma_ap_values[p].reshape(-1, 1), \
                            sigma_aa_values[p].reshape(-1, 1), \
                            sigma_dp_values[p].reshape(-1, 1), \
                            sigma_af_values[p].reshape(-1, 1), \
                            sigma_deltap_values[p].reshape(-1, 1), \
                            sigma_v_values[p].reshape(-1, 1), \
                            C20_values[p].reshape(-1, 1), \
                            C21_values[p].reshape(-1, 1), \
                            C40_values[p].reshape(-1, 1), \
                            C41_values[p].reshape(-1, 1), \
                            C42_values[p].reshape(-1, 1), \
                            C60_values[p].reshape(-1, 1), \
                            C61_values[p].reshape(-1, 1), \
                            C62_values[p].reshape(-1, 1), \
                            C63_values[p].reshape(-1, 1), \
                            C80_values[p].reshape(-1, 1), \
                            C81_values[p].reshape(-1, 1), \
                            v20_values[p].reshape(-1, 1), 
                            v30_values[p].reshape(-1, 1), \
                            kurtosis_values[p].reshape(-1, 1), \
                            kurtosis_f_values[p].reshape(-1, 1), \
                            beta_values[p].reshape(-1, 1), \
                            skewness_values[p].reshape(-1, 1), \
                            skewness_f_values[p].reshape(-1, 1), \
                            mean_amp_values[p].reshape(-1, 1), \
                            std_amp_values[p].reshape(-1, 1), \
                            mean_temp_values[p].reshape(-1, 1), \
                            std_temp_values[p].reshape(-1, 1), \
                            gamma_max_values[p].reshape(-1, 1), \
                            gamma_2_values[p].reshape(-1, 1), \
                            rms_ratio_values[p].reshape(-1, 1), \
                            avg_ratio_values[p].reshape(-1, 1)  ,\
                            spectral_var_values[p].reshape(-1, 1) ,\
                            deviation_values[p].reshape(-1, 1) \
                           ))

        X_all = np.vstack((X_all, X_data))

    return X_all

**createY** expresses the modulation type of each signal example in one-hot encoding format and with an int value between 0 and 6. 
<br><br>
PARAMETERS:
    <br>- **idx**: Indexes of the signal examples that will be used. This parameter is just used to know the amount of signal examples of each combination of moulation type and SNR value are being used.
    <br>- **X_mods**: List of modulation types that are being used.
    <br>- **usedSnr**: List of SNR values that are being used.
<br><br>
RETURNS:
    <br>- **Y**: Modulation types expressed in one-hot encoding format.
    <br>- **Y_Classes**: Modulation types expressed with an int number.

In [ ]:
def createY(num, mods, usedSnr):
    Y_Classes = []
    num_examples = num
    num_snr = len(usedSnr)

    for mod_indx in range(0, len(mods)):
        Y_Classes = Y_Classes + [mod_indx for i in range (num_examples * num_snr)]

    def to_onehot(yy):
        yy1 = np.zeros([len(yy), max(yy)+1])
        yy1[np.arange(len(yy)),yy] = 1
        return yy1

    Y = to_onehot(Y_Classes)
    
    return Y, Y_Classes


Each combination of SNR value and modulation type has 1000 signal examples (**n_examples**), we will choose 400 (**n_train**) of these 1000 signals to be used for training and validating the model. For this reason we only use 400 for the feature selection too.

In [ ]:
n_examples = 1000

n_train = 400

We generate an array called **Y_trainClasses** that contains the modulation type of each signal example in the format of int numbers. 
<br>
The function generateAllDataArray generates an array that has the features that correspond to different signals. These signals are organized first by modulation type and then, by SNR value. For this reason, we only need the information of how many signal examples of each combination of SNR value and modulation type are considered, and how many SNR values are used to generate the corresponding output labels of these signals.

In [ ]:
Y_train, Y_trainClasses = createY(n_train, usedMods, usedSnr)

len(Y_trainClasses)

In [ ]:
features = ['sigma_a','sigma_ap', 'sigma_aa', 'sigma_dp','sigma_af', \
            'sigma_deltap', 'sigma_v', 'C20', 'C21', 'C40', 'C41', \
            'C42','C60', 'C61', 'C62', 'C63', 'C80', 'C81', 'v20', \
            'v30', 'kurtosis', 'kurtosis_f', 'beta', 'skewness', \
            'skewness_f','mean_amp', 'std_amp', 'mean_temp', 'std_temp', \
            'gamma_max', 'gamma_2', 'rms_ratio','avg_ratio', \
            'spectral_var', 'deviation']
len(features)

The selection of the features that are used to discriminate
between the seven modulation schemes is performed by the
application of the **Maximum Relevance Minimum Redundancy (mRMR)**
criterion of the mutual information algorithm.
<br>
We make use of the package mrmr_selection, located at https://github.com/smazzanti/mrmr, 
written by Samuele Mazzanti.
<br>
We repeat the selection of the 9 most important features **iterations** times, using a different group of **n_train** signal examples of each combination of modulation type and SNR value.

In [ ]:
k = 9
iterations = 100

selected_features = []

for n_it in range(iterations):
    print(f'Iteration: {n_it} / {iterations}')
    train_idx = np.random.choice(range(0, n_examples), size = n_train, replace = False)
    X_data = generateAllDataArray(X, train_idx, usedMods, X_mods, lbl, usedSnr)
    
    data = pd.DataFrame(columns = list(map(lambda i :features[i], range(len(features)))), data = list(map(lambda j: X_data[j,:], range(X_data.shape[0]))))
    
    selected_features.append(mrmr_classif(data, pd.Series(Y_trainClasses), K = k))
    

We generate a bar-plot to visualize how many times each feature was selected as one of the 9 most relevant ones.

In [ ]:
y_feat = []
for n_feat, feat in enumerate(features):
    idx_feat = np.where(np.array(selected_features) == feat)[0]
    y_feat.append(len(idx_feat)) 
    
plt.figure(figsize = (10, 4))
plt.bar(features,y_feat)
plt.xticks(rotation = 75,fontsize = 7)
plt.grid(True)
plt.title(f'Feature Selection');


We generate 9 bar-plots to visualize how many times each feature was selected with a specific order of relevance.

In [ ]:
for i in range(len(selected_features)):
    y_feat = []
    for n_feat, feat in enumerate(features):
        idx_feat = np.where(np.array(selected_features[i]) == feat)[0]
        y_feat.append(len(idx_feat)) 
        
    plt.figure(figsize = (10, 4))
    plt.bar(features,y_feat)
    plt.xticks(rotation = 75,fontsize = 7)
    plt.grid(True)
    plt.title(f'Feature {i + 1}')
